In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox
import pandas as pd
import itertools
import matplotlib.pyplot as plt
import tkinter as tk
import tkinter.font as tkFont
import matplotlib.pyplot as plt

plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

# 점수 데이터 불러오기
scores_df = pd.read_excel("player_position_scores_final.xlsx", index_col=0)
scores_df.columns = scores_df.columns.str.strip()
scores_df.index = scores_df.index.str.strip()
print("데이터 컬럼:", scores_df.columns.tolist())
player_names = scores_df.index.tolist()

# REQUIRED 포지션 구성: 각 팀은 5명이어야 하며, 탱커 1, 딜러 2, 힐러 2 이어야 함.
REQUIRED = {"탱커": 1, "딜러": 2, "힐러": 2}

# GUI 시작
root = tk.Tk()
root.title("OW 팀 매칭 (수동 팀 구성 및 자동 매칭)")

frame = tk.Frame(root)
frame.pack(padx=10, pady=10)

# 포지션 목록
POSITIONS = ["탱커", "딜러", "힐러"]
# 플레이어 입력 항목 (10명)
players = []

# 헤더: 이름, 포지션, 점수, 팀 선택
header = tk.Frame(frame)
header.pack()
tk.Label(header, text="이름", width=15).pack(side="left")
tk.Label(header, text="포지션", width=10).pack(side="left")
tk.Label(header, text="점수", width=10).pack(side="left")
tk.Label(header, text="팀", width=10).pack(side="left")

# 점수 업데이트 함수: 입력된 이름과 포지션을 바탕으로 scores_df에서 점수를 읽어옴
def update_score(index):
    def inner(*args):
        name = players[index]["name"].get().strip()
        pos = players[index]["pos"].get().strip()
        if name in scores_df.index and pos in POSITIONS:
            raw_score = scores_df.loc[name, pos]
            scaled_score = round(raw_score, 2)
            players[index]["score"].set(f"{scaled_score}")
        else:
            players[index]["score"].set("")
        update_team_requirements()  # 팀 선택/포지션 변경 시 팀 현황 업데이트 및 검증
    return inner

# 팀 선택 변경 시 호출되는 함수 (팀 콤보박스 변경 시 바로 업데이트)
def update_team_selection(index):
    def inner(*args):
        update_team_requirements()
    return inner

# 10명 입력 필드 생성 (각 행에 이름, 포지션, 점수, 그리고 "팀" 콤보박스 추가)
for i in range(10):
    row = tk.Frame(frame)
    row.pack(pady=2)
    
    name_var = tk.StringVar()
    name_box = ttk.Combobox(row, textvariable=name_var, values=player_names, width=15, state="normal")
    name_box.pack(side="left")
    
    pos_var = tk.StringVar()
    pos_box = ttk.Combobox(row, textvariable=pos_var, values=POSITIONS, width=10, state="readonly")
    pos_box.pack(side="left")
    
    score_var = tk.StringVar()
    score_label = tk.Label(row, textvariable=score_var, width=10)
    score_label.pack(side="left")
    
    # 팀 선택 옵션: "None", "Team 1", "Team 2"
    team_var = tk.StringVar(value="None")
    team_box = ttk.Combobox(row, textvariable=team_var, values=["None", "Team 1", "Team 2"], width=10, state="readonly")
    team_box.pack(side="left")
    
    players.append({"name": name_var, "pos": pos_var, "score": score_var, "team": team_var})
    
    name_var.trace_add("write", update_score(i))
    pos_var.trace_add("write", update_score(i))
    team_var.trace_add("write", update_team_selection(i))

# 실시간 팀 구성 현황을 표시할 레이블 (선택 사항)
team1_label = tk.Label(root, text="Team 1 - 남은: 탱커 1, 딜러 2, 힐러 2")
team1_label.pack(pady=5)
team2_label = tk.Label(root, text="Team 2 - 남은: 탱커 1, 딜러 2, 힐러 2")
team2_label.pack(pady=5)

# 팀 구성 요구사항 업데이트 및 검증 함수
def update_team_requirements():
    # 각 팀의 남은 포지션 수는 REQUIRED를 복사한 후 진행
    remaining_team1 = REQUIRED.copy()
    remaining_team2 = REQUIRED.copy()
    
    for p in players:
        team = p["team"].get().strip()
        pos = p["pos"].get().strip()
        if pos in POSITIONS:
            if team == "Team 1":
                remaining_team1[pos] -= 1
            elif team == "Team 2":
                remaining_team2[pos] -= 1
    
    # 만약 남은 수가 음수이면 초과된 것임 -> 경고 메시지 즉시 표시
    error_msg = ""
    for role, count in remaining_team1.items():
        if count < 0:
            error_msg += f"Team 1에 {role}이 허용 수보다 많습니다.\n"
    for role, count in remaining_team2.items():
        if count < 0:
            error_msg += f"Team 2에 {role}이 허용 수보다 많습니다.\n"
    if error_msg:
        messagebox.showerror("팀 구성 오류", error_msg)
    # 레이블 업데이트
    team1_text = f"Team 1 - 남은: 탱커 {remaining_team1['탱커']}, 딜러 {remaining_team1['딜러']}, 힐러 {remaining_team1['힐러']}"
    team2_text = f"Team 2 - 남은: 탱커 {remaining_team2['탱커']}, 딜러 {remaining_team2['딜러']}, 힐러 {remaining_team2['힐러']}"
    team1_label.config(text=team1_text)
    team2_label.config(text=team2_text)

# 디버깅용 get_score: scores_df에서 점수를 읽어옴
def get_score(name, pos):
    try:
        score = scores_df.loc[name, pos]
        print(f"[DEBUG] get_score - 이름: {name}, 포지션: {pos}, 점수: {score}")
        return score
    except Exception as e:
        print(f"[ERROR] get_score - 이름: {name}, 포지션: {pos}, 에러: {e}")
        return 0.0

def team_score(team):
    return sum([get_score(p["name"], p["pos"]) for p in team])

def predict_win_rate(norm_a, norm_b):
    if norm_a + norm_b == 0:
        return 0.5
    return round(norm_a / (norm_a + norm_b), 3)

# 수동 매칭용 유효성 검사 함수 (팀 구성 정보는 이미 문자열 형태)
def valid_team_str(team):
    roles = [p["pos"].strip() for p in team]
    if len(team) != 5:
        return False
    # 만약 특정 포지션 수가 허용 수를 초과하면 False
    for role, allowed in REQUIRED.items():
        if roles.count(role) > allowed:
            return False
    return (roles.count("탱커") == REQUIRED["탱커"] and 
            roles.count("딜러") == REQUIRED["딜러"] and 
            roles.count("힐러") == REQUIRED["힐러"])

# 기존 자동 매칭 기능 (버튼들은 그대로 유지)
def run_matching(mode="score"):
    input_players = [{"name": p["name"].get().strip(), "pos": p["pos"].get().strip()} for p in players]
    if not all(p["name"] and p["pos"] in POSITIONS for p in input_players):
        messagebox.showerror("입력 오류", "모든 플레이어의 이름과 포지션을 정확히 입력하세요.")
        return

    best_metric = float('inf')
    best_teams = (None, None)
    best_rate = 0

    for combo in itertools.combinations(input_players, 5):
        team_a = list(combo)
        team_b = [p for p in input_players if p not in team_a]
        # 자동 매칭은 입력된 문자열 정보를 사용
        roles_a = [p["pos"] for p in team_a]
        roles_b = [p["pos"] for p in team_b]
        if len(team_a) != 5 or len(team_b) != 5:
            continue
        if roles_a.count("탱커") != REQUIRED["탱커"] or roles_a.count("딜러") != REQUIRED["딜러"] or roles_a.count("힐러") != REQUIRED["힐러"]:
            continue
        if roles_b.count("탱커") != REQUIRED["탱커"] or roles_b.count("딜러") != REQUIRED["딜러"] or roles_b.count("힐러") != REQUIRED["힐러"]:
            continue

        team_score_a = team_score(team_a)
        team_score_b = team_score(team_b)
        avg_a = team_score_a / 5
        avg_b = team_score_b / 5
        norm_a = avg_a / 5000
        norm_b = avg_b / 5000
        rate = predict_win_rate(norm_a, norm_b)
        print(f"[DEBUG] 팀 구성: {team_a} / {team_b}")
        print(f"[DEBUG] 팀 A 합계: {team_score_a}, 팀 B 합계: {team_score_b}")
        print(f"[DEBUG] 팀 A 평균: {avg_a}, 팀 B 평균: {avg_b}")
        print(f"[DEBUG] norm_A: {norm_a}, norm_B: {norm_b}, 예상 승률: {rate}")
        print("---------------------------------------------------------")
        metric = abs(rate - 0.5) if mode=="winrate" else abs(team_score_a - team_score_b)
        if metric < best_metric:
            best_metric = metric
            best_teams = (team_a, team_b)
            best_rate = rate

    if best_teams[0] is None:
        messagebox.showerror("매칭 실패", "탱커 1/딜러 2/힐러 2 조건을 만족하는 팀 구성이 없습니다.")
        return

    names_a = ", ".join([p["name"] for p in best_teams[0]])
    names_b = ", ".join([p["name"] for p in best_teams[1]])
    
    final_team_score_a = team_score(best_teams[0])
    final_team_score_b = team_score(best_teams[1])
    final_avg_a = final_team_score_a / 5
    final_avg_b = final_team_score_b / 5
    teamA_win_rate = best_rate
    teamB_win_rate = 1 - best_rate

    plt.bar(["Team A", "Team B"], [final_team_score_a, final_team_score_b], color=["blue", "orange"])
    plt.title("팀 점수 합계 (원래 스케일)")
    plt.ylabel("점수")
    plt.show(block=True)
    
    messagebox.showinfo(
        "매칭 결과",
        f"Team A: {names_a}\nTeam B: {names_b}\n\n"
        f"Team A 평균 점수: {round(final_avg_a,2)}\n"
        f"Team B 평균 점수: {round(final_avg_b,2)}\n\n"
        f"예상 승률 (Team A 기준): {round(teamA_win_rate*100,1)}%\n"
        f"예상 승률 (Team B 기준): {round(teamB_win_rate*100,1)}%"
    )

def run_matching_score():
    run_matching(mode="score")

def run_matching_winrate():
    run_matching(mode="winrate")

def run_top_winrate_matches():
    input_players = [{"name": p["name"].get().strip(), "pos": p["pos"].get().strip()} for p in players]
    if not all(p["name"] and p["pos"] in POSITIONS for p in input_players):
        messagebox.showerror("입력 오류", "모든 플레이어의 이름과 포지션을 정확히 입력하세요.")
        return
    results = []
    for combo in itertools.combinations(input_players, 5):
        team_a = list(combo)
        team_b = [p for p in input_players if p not in team_a]
        if len(team_a) != 5 or len(team_b) != 5:
            continue
        roles_a = [p["pos"] for p in team_a]
        roles_b = [p["pos"] for p in team_b]
        if roles_a.count("탱커") != REQUIRED["탱커"] or roles_a.count("딜러") != REQUIRED["딜러"] or roles_a.count("힐러") != REQUIRED["힐러"]:
            continue
        if roles_b.count("탱커") != REQUIRED["탱커"] or roles_b.count("딜러") != REQUIRED["딜러"] or roles_b.count("힐러") != REQUIRED["힐러"]:
            continue
        score_a = team_score(team_a)
        score_b = team_score(team_b)
        avg_a = score_a / 5
        avg_b = score_b / 5
        norm_a = avg_a / 5000
        norm_b = avg_b / 5000
        winrate = predict_win_rate(norm_a, norm_b)
        results.append((abs(winrate - 0.5), winrate, team_a, team_b))
    if not results:
        messagebox.showerror("매칭 실패", "탱커 1/딜러 2/힐러 2 조건을 만족하는 팀 구성이 없습니다.")
        return
    results.sort(key=lambda x: x[0])
    top_results = results[:3]
    msg = ""
    for idx, (_, winrate, team_a, team_b) in enumerate(top_results, 1):
        names_a = ", ".join([p["name"] for p in team_a])
        names_b = ", ".join([p["name"] for p in team_b])
        msg += f"[{idx}] 예상 승률: {round(winrate*100,1)}%\n"
        msg += f"Team A: {names_a}\nTeam B: {names_b}\n\n"
    messagebox.showinfo("승률 기준 상위 추천 매칭", msg)

# 기존 버튼들 (자동 매칭 기능은 그대로 유지)
tk.Button(root, text="⚖ 점수 기반 최적 매칭", command=run_matching_score).pack(pady=5)
tk.Button(root, text="🎯 승률 50% 기반 매칭", command=run_matching_winrate).pack(pady=5)
tk.Button(root, text="⭐ 승률 기준 상위 추천 (TOP 3)", command=run_top_winrate_matches).pack(pady=5)

# 추가 버튼: 수동 팀 구성 – 사용자가 팀 콤보박스를 통해 팀을 할당한 상태에서 실행
def run_manual_match():
    team_a = [{"name": p["name"].get().strip(), "pos": p["pos"].get().strip()} 
              for p in players if p["team"].get().strip() == "Team 1"]
    team_b = [{"name": p["name"].get().strip(), "pos": p["pos"].get().strip()} 
              for p in players if p["team"].get().strip() == "Team 2"]
    
    # 각 팀에 정확히 5명이 있어야 함
    if len(team_a) != 5 or len(team_b) != 5:
        messagebox.showerror("팀 구성 오류", "각 팀은 정확히 5명이어야 합니다.")
        return

    # 문자열 기반 유효성 검사 함수 사용
    if not valid_team_str(team_a) or not valid_team_str(team_b):
        messagebox.showerror("팀 구성 오류", "각 팀은 탱커 1, 딜러 2, 힐러 2의 구성이 되어야 합니다.\n(포지션 갯수가 초과되거나 부족하면 매칭할 수 없습니다.)")
        return

    team_score_a = sum([get_score(p["name"], p["pos"]) for p in team_a])
    team_score_b = sum([get_score(p["name"], p["pos"]) for p in team_b])
    avg_a = team_score_a / 5
    avg_b = team_score_b / 5
    norm_a = avg_a / 5000
    norm_b = avg_b / 5000
    teamA_win_rate = predict_win_rate(norm_a, norm_b)
    teamB_win_rate = 1 - teamA_win_rate
    print(f"[DEBUG] 수동 매칭 - Team A 점수 합계: {team_score_a}, Team B 점수 합계: {team_score_b}")
    print(f"[DEBUG] 수동 매칭 - Team A 평균: {avg_a}, Team B 평균: {avg_b}")
    print(f"[DEBUG] 수동 매칭 - norm_A: {norm_a}, norm_B: {norm_b}")
    print(f"[DEBUG] 수동 매칭 - Team A 승률: {teamA_win_rate}, Team B 승률: {teamB_win_rate}")
    
    plt.bar(["Team 1", "Team 2"], [team_score_a, team_score_b], color=["blue", "orange"])
    plt.title("수동 팀 구성 - 팀 점수 합계")
    plt.ylabel("점수")
    plt.show(block=True)
    
    messagebox.showinfo(
        "수동 팀 구성 결과",
        f"Team 1: {', '.join([p['name'] for p in team_a])}\n"
        f"Team 2: {', '.join([p['name'] for p in team_b])}\n\n"
        f"Team 1 평균 점수: {round(avg_a,2)}\n"
        f"Team 2 평균 점수: {round(avg_b,2)}\n\n"
        f"예상 승률 (Team 1 기준): {round(teamA_win_rate*100,1)}%\n"
        f"예상 승률 (Team 2 기준): {round(teamB_win_rate*100,1)}%"
    )

tk.Button(root, text="🎯 수동 팀 구성", command=run_manual_match).pack(pady=5)

root.mainloop()
